In [4]:
cd tencent_algo_2020/

[Errno 2] No such file or directory: 'tencent_algo_2020/'
/home/sayhi/workspaces/tencent_algo_2020


In [5]:
import time
import data
import lightgbm as lgb
import numpy as np
import os
import sys
import re
import pandas as pd
import matplotlib
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score
from gensim.models import word2vec, keyedvectors
import logging

In [6]:
train_ad, train_click, train_user, test_ad, test_click = data.load_data()
# train_record
train_record = pd.merge(train_click, train_ad, on="creative_id")
# test_record
test_record = pd.merge(test_click, test_ad, on="creative_id")

train_grouped = train_record.groupby("user_id")
test_grouped = test_record.groupby("user_id")

In [ ]:
list_train = train_grouped.agg(list).reset()

In [12]:
count = train_grouped.count()["ad_id"]

In [19]:
count.sort_values()

user_id
592868        2
306941        3
229147        3
196804        3
697730        3
          ...  
549175     1273
178333     1321
677301     1661
157585     1706
839368    16868
Name: ad_id, Length: 900000, dtype: int64

In [34]:
(count > 400).sum()

271

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline

In [3]:
train_root = "../dataset/train/"
train_user_path = os.path.join(train_root, "user.csv")
train_ad_path = os.path.join(train_root, "ad.csv")
train_click_path = os.path.join(train_root, "click_log.csv")

test_root = "../dataset/test/"
test_ad_path = os.path.join(test_root, "ad.csv")
test_click_path = os.path.join(test_root, "click_log.csv")

In [4]:
train_ad = pd.read_csv(train_ad_path, na_values="\\N")
train_click = pd.read_csv(train_click_path, na_values="\\N")
train_user = pd.read_csv(train_user_path, na_values="\\N")

test_ad = pd.read_csv(test_ad_path, na_values="\\N")
test_click = pd.read_csv(test_click_path, na_values="\\N")

In [5]:
train_click
test_click

time  user_id  creative_id  click_times
0            9    30920       567330            1
1           65    30920      3072255            1
2           56    30920      2361327            1
3            6   309204       325532            1
4           59   309204      2746730            1
...        ...      ...          ...          ...
30082766     6    30920       228129            1
30082767    57    30920      2361397            1
30082768    26    30920       629802            1
30082769    86    30920      2713031            1
30082770    25    30920       629802            1

[30082771 rows x 4 columns]

time  user_id  creative_id  click_times
0           20  3131989       645764            1
1           20  3131989      1027422            1
2           20  3131989      1106443            1
3           20  3131989       629802            1
4           59  3131989      2839769            1
...        ...      ...          ...          ...
33585507    16  3131989       585914            1
33585508    13  3131989       749134            1
33585509    12  3131989       589027            1
33585510    61  3131989      2404831            1
33585511    14  3131989       665822            1

[33585512 rows x 4 columns]

In [6]:
train_creative_id = train_click["creative_id"]
test_creative_id = test_click["creative_id"]
train_unique_creative = train_creative_id.unique()
test_unique_creative = test_creative_id.unique()

intersection = np.intersect1d(train_unique_creative, test_unique_creative)

# train和test中重复的素材id
print(intersection.shape)
# train和test中各自的素材id数量
print(train_unique_creative.shape, test_unique_creative.shape)

(1686522,)
(2481135,) (2618159,)


In [10]:
print(train_click.count(), test_click.count())

tr_c_times = train_click["click_times"]
tr_c_times.value_counts()
train_click[tr_c_times == 152]

te_c_times = test_click["click_times"]
te_c_times.value_counts()

time           30082771
user_id        30082771
creative_id    30082771
click_times    30082771
dtype: int64 time           33585512
user_id        33585512
creative_id    33585512
click_times    33585512
dtype: int64


1      28426543
2       1507449
3         90356
4         43047
5          7507
6          3512
7          1529
8          1486
9           452
10          270
11          172
12          116
13           76
14           62
15           36
16           34
17           22
18           21
21           17
20           13
19           10
22            5
23            4
26            3
32            3
27            3
30            3
24            3
25            2
29            2
33            2
37            2
28            1
67            1
34            1
35            1
36            1
43            1
44            1
50            1
152           1
Name: click_times, dtype: int64

time  user_id  creative_id  click_times
2212918    10   428573       593214          152

1      31695705
2       1706687
3        108696
4         51499
5         10133
         ...   
167           1
44            1
69            1
60            1
185           1
Name: click_times, Length: 93, dtype: int64

In [137]:
train_user_id = train_user["user_id"]
print(train_user_id)
test_user_id = test_click["user_id"]
print(test_user_id.unique().shape)

print(train_user_id.min(), train_user_id.max(), test_user_id.min(), test_user_id.max())

0              1
1              2
2              3
3              4
4              5
           ...  
899995    899996
899996    899997
899997    899998
899998    899999
899999    900000
Name: user_id, Length: 900000, dtype: int64
(1000000,)
1 900000 3000001 4000000


In [119]:
train_ad.isnull().sum()

creative_id              0
ad_id                    0
product_id          929524
product_category         0
advertiser_id            0
industry            101048
dtype: int64

In [ ]:
nums_ad = train_ad["product_id"].count()
print("total numbers of ad is {}".format(nums_ad))

In [12]:
train_ad.head()

creative_id  ad_id  product_id  product_category  advertiser_id  industry
0            1      1         NaN                 5            381      78.0
1            4      4         NaN                 5            108     202.0
2            7      7         NaN                 5            148     297.0
3            8      8         NaN                 5            713     213.0
4            9      9         NaN                 5            695     213.0

In [103]:
train_ad.describe()
train_ad.info()
train_ad.shape
train_ad.isnull().sum()

creative_id         ad_id  product_category  advertiser_id
count  2.481135e+06  2.481135e+06      2.481135e+06   2.481135e+06
mean   2.192008e+06  1.888107e+06      7.805932e+00   2.741624e+04
std    1.285497e+06  1.099764e+06      6.770557e+00   1.534631e+04
min    1.000000e+00  1.000000e+00      1.000000e+00   2.000000e+00
25%    1.076370e+06  9.391925e+05      2.000000e+00   1.497400e+04
50%    2.191948e+06  1.889017e+06      5.000000e+00   2.519700e+04
75%    3.304420e+06  2.839458e+06      1.800000e+01   3.825400e+04
max    4.445718e+06  3.812200e+06      1.800000e+01   6.296500e+04

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2481135 entries, 0 to 2481134
Data columns (total 6 columns):
creative_id         int64
ad_id               int64
product_id          object
product_category    int64
advertiser_id       int64
industry            object
dtypes: int64(4), object(2)
memory usage: 113.6+ MB


(2481135, 6)

creative_id         0
ad_id               0
product_id          0
product_category    0
advertiser_id       0
industry            0
dtype: int64

In [53]:
(train_ad != "\\N").sum()

creative_id         2481135
ad_id               2481135
product_id          1551611
product_category    2481135
advertiser_id       2481135
industry            2380087
dtype: int64

In [68]:
# #product_id 中缺失值的比例
miss_pid = (train_ad["product_id"] != "\\N").sum()
print("In train data, product id have {} / {}, exist {:.2f}%, miss {:.2f}%".format(miss_pid, nums_ad, miss_pid/nums_ad*100, (1-miss_pid/nums_ad)*100))

In train data, product id have 1551611 / 2481135, exist 62.54%, miss 37.46%


In [84]:
# #industry 中缺失值的比例
miss_pid = (train_ad["industry"] != "\\N").sum()
print("In train data, industry have {} / {}, exist {:.2f}%, miss {:.2f}%".format(miss_pid, nums_ad, miss_pid/nums_ad*100, (1-miss_pid/nums_ad)*100))


# industry 列中max是335，min是1，一共是325，有缺失值
col_industry = train_ad["industry"]
filter_industry = col_industry[train_ad["industry"] != "\\N"].astype(int)


In train data, industry have 2380087 / 2481135, exist 95.93%, miss 4.07%


In [101]:
train_user = pd.read_csv(train_user_path)
print(train_user.head())

   user_id  age  gender
0        1    4       1
1        2   10       1
2        3    7       2
3        4    5       1
4        5    4       1


In [16]:
train_user

user_id  age  gender
0             1    4       1
1             2   10       1
2             3    7       2
3             4    5       1
4             5    4       1
...         ...  ...     ...
899995   899996    5       1
899996   899997    3       2
899997   899998    4       2
899998   899999    3       1
899999   900000    3       2

[900000 rows x 3 columns]

In [19]:
train_user["age"].value_counts()
train_user["gender"].value_counts()

3     202909
4     150578
2     149271
5     130667
6     101720
7      66711
1      35195
8      31967
9      19474
10     11508
Name: age, dtype: int64

1    602610
2    297390
Name: gender, dtype: int64